# Part 1

# Ingest the TSV file into a DataFrame

In [1]:
# Import the necessary libraries
import pandas as pd

In [2]:
# Load the tsv DATASET
df = pd.read_csv('correct_twitter_201904.tsv', sep='\t')

In [3]:
df.head()

,id,event,ts1,ts2,from_stream,directly_from_stream,from_search,directly_from_search,from_quote_search,directly_from_quote_search,...,retweeted,retweeted_author_id,retweeted_handle,retweeted_follower_count,mentioned_author_ids,mentioned_handles,hashtags,urls,media_keys,place_id
0,1131594960443199488,britney_201904,2022-02-28 09:34:44.627023-05:00,2022-02-28 09:34:44.627023-05:00,True,True,False,False,False,False,...,1.130918e+18,3.042894e+09,Iesbwian,22760.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1131594976750653440,britney_201904,2022-02-28 09:34:44.626921-05:00,2022-02-28 09:34:44.626921-05:00,True,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1131589737955942405,britney_201904,2022-02-28 09:34:44.634058-05:00,2022-02-28 09:34:44.634058-05:00,True,True,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1131594909469892610,britney_201904,2022-02-28 09:34:44.627125-05:00,2022-02-28 09:34:44.627125-05:00,True,True,False,False,False,False,...,1.130918e+18,3.042894e+09,Iesbwian,22760.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1131594812694511617,britney_201904,2022-02-28 09:34:44.627227-05:00,2022-02-28 09:34:44.627227-05:00,True,True,False,False,False,False,...,1.130918e+18,3.042894e+09,Iesbwian,22760.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88037 entries, 0 to 88036
Data columns (total 46 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   id                             88037 non-null  int64  
 1   event                          88037 non-null  object 
 2   ts1                            88037 non-null  object 
 3    ts2                           88037 non-null  object 
 4   from_stream                    88037 non-null  bool   
 5   directly_from_stream           88037 non-null  bool   
 6   from_search                    88037 non-null  bool   
 7   directly_from_search           88037 non-null  bool   
 8   from_quote_search              88037 non-null  bool   
 9   directly_from_quote_search     88037 non-null  bool   
 10  from_convo_search              88037 non-null  bool   
 11  directly_from_convo_search     88037 non-null  bool   
 12  from_timeline_search           88037 non-null 

# Part 2

# Now the data is ready for querying

In [5]:
# Define the term for filtering
term = "Britney Spears"

In [6]:
# Convert 'created_at' column to date time format with UTC timezone
df['created_at'] = pd.to_datetime(df['created_at'], utc=True)

In [7]:
#Filter tweets containing the specified term.
    
def filter_tweets_by_term(df, term):
    return df[df['text'].str.contains(term, case=False, na=False)]

In [8]:
#Count how many tweets containing the term were posted each day.

def tweets_per_day(df, term):
    
    filtered_df = filter_tweets_by_term(df, term)
    filtered_df.loc[:, 'created_at'] = pd.to_datetime(filtered_df['created_at'], errors='coerce')
    return filtered_df.groupby(filtered_df['created_at'].dt.date).size()

In [9]:
#Count how many unique users posted tweets containing the term
def unique_users(df, term):
    filtered_df = filter_tweets_by_term(df, term)
    return filtered_df['id'].nunique()

In [10]:
#Calculate the average number of likes for tweets containing the term
def average_likes(df, term):
    filtered_df = filter_tweets_by_term(df, term)
    return filtered_df['like_count'].mean()

In [11]:
#Get the place IDs where tweets containing the term were posted.
def place_ids(df, term):
    filtered_df = filter_tweets_by_term(df, term)
    return filtered_df['place_id'].value_counts()

In [12]:
# Determine the number of tweets containing the term posted each hour of the day.
def tweets_per_hour(df, term):
    filtered_df = filter_tweets_by_term(df, term)
    
    # Ensure 'created_at' is in datetime format using .loc
    filtered_df.loc[:, 'created_at'] = pd.to_datetime(filtered_df['created_at'], errors='coerce')
    
    # Group by the hour of the day (rounded down) and count the number of tweets
    return filtered_df.groupby(filtered_df['created_at'].dt.floor('h')).size()

In [13]:
# Find the user who posted the most tweets containing the term
def most_active_user(df, term):
    filtered_df = filter_tweets_by_term(df, term)
    return filtered_df['author_handle'].value_counts().idxmax()

1.How many tweets were posted containing the term on each day?

In [17]:
print(tweets_per_day(df, term))

created_at
2019-03-01       1
2019-03-02       1
2019-03-03       3
2019-03-04       1
2019-03-05       3
              ... 
2019-05-27     862
2019-05-28    1500
2019-05-29    1749
2019-05-30    1730
2019-05-31     336
Length: 77, dtype: int64


 2.How many unique users posted a tweet containing the term?

In [16]:
print(unique_users(df, term))

72849


3.How many likes did tweets containing the term get, on average?

In [18]:
print(average_likes(df, term))

6.023088855028895


4.Where (in terms of place IDs) did the tweets come from?

In [19]:
print(place_ids(df, term))

place_id
3b77caf94bfc81fe    37
1ef1183ed7056dc1    17
06b9691f34c91d1c    11
1d9a5370a355ab0c    10
01a9a39529b27f36    10
                    ..
5a23da841b5691c7     1
d838baf51d1c629d     1
46c1b1ab24d7e11a     1
5f73d127c7ccf9d9     1
019634185ded88a2     1
Name: count, Length: 520, dtype: int64


5.What times of day were the tweets posted at?

In [20]:
print(tweets_per_hour(df, term))

created_at
2019-03-01 17:00:00+00:00      1
2019-03-02 02:00:00+00:00      1
2019-03-03 06:00:00+00:00      1
2019-03-03 19:00:00+00:00      1
2019-03-03 20:00:00+00:00      1
                            ... 
2019-05-30 23:00:00+00:00     81
2019-05-31 00:00:00+00:00     75
2019-05-31 01:00:00+00:00    101
2019-05-31 02:00:00+00:00     87
2019-05-31 03:00:00+00:00     73
Length: 919, dtype: int64


6.Which user posted the most tweets containing the term?

In [21]:
print(most_active_user(df, term))

Christo45998808
